## Graph construction

Our graph should be represented as an adjacency matrix and at the same times capture the interaction between $u_i$ and $i_j$. Initially we have an user-item interaction matrix $R$ that does capture the $u_i$ and $i_j$ interaction. Later we need to construct an adjacency matrix $A$ from $R$. We can achieve it by the following

$$A = \begin{pmatrix}0 & R \\ R^T & 0\end{pmatrix}$$

## Preprocess

In [ ]:
ZIP_PATH = "../data/raw/ml-100k.zip"
# extract zip

## Model